In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time

%matplotlib widget

In [2]:
from bluesky_tutorial_utils import setup_data_saving
from bluesky import RunEngine
from bluesky.plans import count
from utils import simple_integration

RE = RunEngine()
catalog = setup_data_saving(RE)

In [3]:
from simulated_hardware import detector, shutter, load_sample, light, history_reset, sim_sleep

In [8]:
from simulated_hardware import detector, load_sample, light, sim_sleep

## Intro text

In [4]:
#load sample
RE(load_sample(1))

#turn on light
RE(light(True))

#count
RE(count([detector]))


('0a048642-ed55-482a-872a-ebc45a1690ef',)

In [5]:
#retrieve image from databroker
my_im = catalog[-1].primary.read().detector_image[0]
#plot
plt.figure()
plt.imshow(my_im)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Dark image introduction

In [6]:
#we started at t=0, so we'll just go before then to measure the dark.
#time_travel(-1)

RE(light(False))

sim_sleep(10)

RE(count([detector]))

my_dark = catalog[-1].primary.read().detector_image[0]

sleeping for 10


In [7]:
plt.figure()
plt.imshow(my_dark)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.figure()
plt.imshow(my_im - my_dark)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Integrate to get pattern

In [9]:
intensity = simple_integration(my_im - my_dark,num_bins=301)
#x = np.linspace(0, 30, num=301)


plt.figure()
plt.plot(intensity, label="'measured'")
plt.legend()
#plt.gcf()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Bluesky plan to gather light/dark automatically

In [10]:
def dark_light_subtract(sample_num):
    #close shutter if not already closed
    yield from light(False)
        
    #move to desired sample
    yield from load_sample(sample_num)
    
    #take dark image
    yield from count([detector])
    
    #open shutter
    yield from light(True)
    
    #take light image
    yield from count([detector])
    
    #close shutter to be nice to detector
    yield from light(False)
    

def process_data(pair):
    #assuming pair is tuple
    my_dark = catalog[pair[0]].primary.read().detector_image[0]
    my_light = catalog[pair[1]].primary.read().detector_image[0]
    
    this_int = simple_integration(my_light - my_dark,num_bins=301)
    
    return this_int

### Explain uid pairs in this context.

In [11]:
uids = RE(dark_light_subtract(1))

In [12]:
uids

('111869f7-3a8e-4827-a215-6c9a41a9c576',
 '1f7a2294-914d-4586-ac9a-bda1fae74df8')

In [13]:
this_intensity = process_data(uids)
plt.figure()
plt.plot(this_intensity)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Concluding remarks